In [7]:
def spatial_decomposition(X, unit_cell, *args):
    '''
    % decomposite the spatial domain into cells D with vertices V,
    %
    % Output
    %  V - list of vertices
    %  F - list of faces
    %  I_FD - incidence matrix between faces to cells

    % compute voronoi decomposition
    % V - list of vertices of the Voronoi cells
    % D   - cell array of Vornoi cells with centers X_D ordered accordingly
    '''
    # Imports
    import getopt
    import numpy as np
    from numpy import cumsum, zeros, unique, sort
    from spatialDecompFunctions import generateUnitCells
    from scipy.spatial import Voronoi
    from scipy.sparse import csr_matrix
    
    if unit_cell.all() == None:
        unit_cell = calcUnitCell(X)
    
    if args[0] == 'unit_cell':
    
        # compute the vertices
        [V, faces] = generateUnitCells(X, unit_cell, args[:])
        
        D = np.empty(len(X),dtype=object)

        for k in range(X.shape[0]):
            D[k] = faces[k, :]
    
    else:    
        var_arg_in = args[0]
        dummyCoordinates = calcBoundary(X, unit_cell, var_arg_in)

        [V,D] = Voronoi(np.array([[X, dummyCoordinates]]).T, 
                        qhull_options = {'Q5','Q6','Qs'}
            ) #,'QbB'

        D = D[1:X.shape[0]]

    # now we need some adjacencies and incidences
    iv = [D[:]]            # nodes incident to cells D
    id = zeros(len(iv))   # number the cells
        
    # Some MATLAB stuff goin on here... : p = np.array([[0, cumsum(cellfun('prodofsize',D))]]).T
    D_prod = 1
    for elem in D.shape:
        D_prod *= elem
    p = np.array( [[0, cumsum(D_prod)]] ).T
    for k in range(len(D)):
        id[p(k)+1 : p(k+1)] = k
        
    # next vertex
    ind_x = list( range(2,len(iv)+1) )
    ind_x[p[2:]] = p[1:-2] + 1
    iv_n = iv(ind_x)

    # edges list
    F = [iv[:], iv_n[:]]

    # should be unique (i.e one edge is incident to two cells D)
    [F, _, ie] = unique(sort(F,axis=1), axis=0)

    # faces incident to cells, F x D
    #I_FD = sparse(ie,id,1);
    I_FD = csr_matrix( (ie, id, 1) ) # could also use csc_matrix() if it improves
                                 # performance !
    

    return V, F, I_FD


# numpy.loadtxt
def calcBoundary(X, unit_cell, var_arg_in='hull'):
    '''
    dummy coordinates so that the voronoi-cells of X are finite

    Inputs:
    --------------
    X : n x 2 numpy array of [x,y] vertex coordinates

    unit_cell : n x 2 numpy array of [x,y] coordinates of "pixel" boundary (e.g. hexagon or square)

    var_arg_in : ???
    ???


    Outputs:
    --------------
    ??? : ???
    ???

    Dependencies:
    --------------
    import getopt
    import numpy as np
    from scipy.spatial import ConvexHull
    from numpy import arctan2, diff, cumsum, matrix, squeeze, unique
    from statistics import mean
    from math import sqrt, floor, copysign
    '''
    # Imports
    import getopt
    import numpy as np
    from scipy.spatial import ConvexHull
    from numpy import arctan2, cumsum, matrix, squeeze, unique, linspace
    from statistics import mean
    from math import sqrt, floor, copysign
    from orix.quaternion.rotation import Rotation
    import orix.vector as vect
    
    from spatialDecompFunctions import householderMatrix, translationMatrix#, erase_linearly_dependent_points

    dummy_coordinates = []

    boundary = str(var_arg_in)

    if boundary.isalpha():

        if (boundary.lower() == 'hull' or
            boundary.lower() == 'convexhull'):
            
            bounding_X  = erase_linearly_dependent_points(X)
            print(bounding_X)
            
        elif boundary.lower() == 'cube':
            # set up a rectangular box
            envelope_X = [np.amin(X), np.amax(X)]
            bounding_X = [
                envelope_X[1], envelope_X[3],
                envelope_X[2], envelope_X[3],
                envelope_X[2], envelope_X[4],
                envelope_X[1], envelope_X[4],
                envelope_X[1], envelope_X[3],
                ]

        else:
            raise ValueError('Unknown boundary type. Available options are \
            ''hull'', ''convexhull'' and ''cube''.')

    elif isinstance(boundary, float):
        bounding_X = boundary
    
    radius = mean( sum( (unit_cell**2) ** 0.5 ) )

    edge_length = ((matlabdiff(bounding_X)**2).sum(axis=1, keepdims=True))**0.5 
    #edgeLength = sqrt(sum(diff(boundingX).^2,2));

    # fill each line segment with nodes every 20 points (in average)
    nto = np.int32(np.fix( (edge_length>0)*4 ))

    #nto = floor( edge_length*(2*radius) )
    #nto = fix((edgeLength>0)*4); fix(edgeLength*(2*radius));

    cs = np.int32(cumsum(np.vstack([1, 1 + nto])))

    bounding_X = left_hand_assignment(bounding_X, cs)


    # interpolation
    for k in range(nto.shape[0]):
        for dim in range(nto.shape[1]):
            #print(nto[k]+2)
            bounding_X[cs[k]:cs[k+1], dim] = linspace(
                bounding_X[cs[k],dim],
                bounding_X[cs[k+1]-1, dim],
                nto[k,dim]+1
            )
    

    # homogeneous coordinates
    X = np.hstack([X, np.ones([X.shape[0],1])])
    bounding_X = np.hstack([bounding_X, np.ones([bounding_X.shape[0],1])])

    
    # direction of the edge
    edge_direction = matlabdiff(bounding_X)
    edge_angle = arctan2( edge_direction[:, 1], edge_direction[:,0] )
    edge_length = np.sqrt( (edge_direction**2).sum(axis = 1, keepdims=True))


    # shift the starting vertex
    r = Rotation.from_axes_angles(np.array([0, 0, 1]), edge_angle)
    a = vect.Vector3d([0, radius, 1])
    b_X = np.squeeze(r * a)
    offset_X = np.squeeze(b_X.data) - np.squeeze(bounding_X[0:-1, :])
    
    

    for k in range(bounding_X.shape[0]-1):
        
        # mirror the point set X on each edge
        p_X = X * -1*( translationMatrix[offset_X[k, :]] * householderMatrix[edge_direction[k, :]] * translationMatrix[offset_X[k, :]] ).T
        
        # distance between original and mirrored point
        dist = sqrt( sum( (X[:,1:2]-p_X[:,1:2])**2, axis = 1 ) )

        intend_X = 2 * radius * copysign(1, edge_direction[k,1:2])
        
        # now try to delete unnecessary points
        m = 2

        condition1 = True
        while condition1:
        
            tmp_X = p_X[dist < m*radius,1:2]

            right = tmp_x[np.tile(tmp_X) - ( bounding_X[k, 0:2]   - intend_X ) * edge_direction[k, 1:2].T < 0]
            left  = tmp_x[np.tile(tmp_X) - ( bounding_X[k+1, 0:2] + intend_X ) * edge_direction[k, 1:2].T < 0]
            
            tmp_X = tmp_X[ not(right or left), :]
            
            if edge_length(k) / tmp_X.shape[0] < radius/3:
                break
            elif m < 2**7:
                m = m*2
            else:
                m = m+10
        
        dummy_coordinates = np.array([[dummy_coordinates, tmp_X]]).T
        
    _, dummy_coordinates, _, _ = unique(dummyCoordinates, axis = 0)

def erase_linearly_dependent_points(X):
    '''
    subfunction to remove linearly dependent points.

    Inputs:
    --------------
    k : ???
        ???

    Outputs:
    --------------
    ??? : ???
        ???

    Dependencies:
    --------------
    from scipy.spatial import ConvexHull
    '''
    from orix.utilities.utilities import regularPoly
    import numpy as np
    from scipy.spatial import ConvexHull
    from orix.utilities.utilities import sortrows
    k = ConvexHull(X)
    k = k.vertices

    # erase all linear dependent points
    angle = np.arctan2(X[k[0:-1],0]-X[k[1:],0],
        X[k[0:-1],1]-X[k[1:],1])
    test = np.abs(np.diff(angle))>np.spacing(1.0)
    k2 = k[np.concatenate([[True], test, [True]])]
    boundingX = X[k2,:]

    boundingX = sortrows(boundingX)
    boundingX  = np.flipud(boundingX )
    boundingX  = np.vstack([boundingX [-1,:], boundingX ])

    return boundingX

def left_hand_assignment(X, a):
    '''
    Attempts to replicate MATLAB left-hand assignment for a 2D array.
 

    Inputs:
    --------------
    X : 2D numpy array
 

    Outputs:
    --------------
    X : 2D numpy array
        This is different from the X passed in, it will return a larger array with
        more zeros.
 
    Dependencies:
    --------------
    import numpy as np
    '''
 
    import numpy as np
    import warnings
    
    if a.dtype != 'int':
        warnings.warn('parameter ''a'' must be of integer type. Converting ''a'' into integers and moving on...')

    a = np.int32(a)
 
    bound_X_init = X
    ncols = X.shape[1]
 
    max_a = np.amax(a)
    bound_X_fin = np.zeros([max_a, ncols])
 
    bound_X_fin[0:bound_X_init.shape[0], :] = bound_X_init
 
    for i, elem in enumerate(a[1:]):
        bound_X_fin[elem-1, :] = bound_X_init[i+1, :]
 
    return bound_X_fin

def matlabdiff(myArray):
    return myArray[1:,:] - myArray[0:-1,:]


In [8]:
import numpy as np
X = np.loadtxt('spatialDecomposition_input_x.csv', delimiter=',', dtype=float)
unit_cell = np.loadtxt('spatialDecomposition_input_unitCell.csv', delimiter=',', dtype=float)
spatial_decomposition(X, unit_cell, 'hull')

[[  0.           0.        ]
 [156.74951098 146.79084799]
 [156.74951098   0.43301135]
 [156.49951176   0.        ]
 [  0.24999922 146.79084799]
 [  0.         146.35783664]
 [  0.           0.        ]]
(31, 3)
(30, 3)
(30,)
[[0.00000000e+00 0.00000000e+00 1.00000000e+00]
 [1.56749511e+02 1.46790848e+02 1.00000000e+00]
 [1.56749511e+02 4.33011351e-01 1.00000000e+00]
 [1.56749511e+02 0.00000000e+00 1.00000000e+00]
 [1.56749511e+02 1.46790848e+02 1.00000000e+00]
 [1.56749511e+02 1.46790848e+02 1.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]
 [3.91873777e+01 0.00000000e+00 1.00000000e+00]
 [7.83747555e+01 0.00000000e+00 1.00000000e+00]
 [1.17562133e+02 0.00000000e+00 1.00000000e+00]
 [1.56749511e+02 4.33011351e-01 1.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]
 [3.91248779e+01 0.00000000e+00 1.00000000e+00]
 [7.82497559e+01 0.00000000e+00 1.00000000e+00]
 [1.17374634e+02 0.00000000e+00 1.00000000e+00]
 [1.56499512e+02 0.00000000e+00 1.00000000e+00]
 [0.00

/var/folders/nz/l5ynytfs6sj2drjd5cs1wckc0000gn/T/ipykernel_63459/1992495348.py:294: UserWarning: parameter a must be of integer type. Converting a into integers and moving on...
  warnings.warn('parameter ''a'' must be of integer type. Converting ''a'' into integers and moving on...')


ValueError: operands could not be broadcast together with shapes (30,3) (30,3,1) 

In [ ]:
import getopt
import numpy as np
from scipy.spatial import ConvexHull
from numpy import arctan2, diff, cumsum, matrix, squeeze, unique, linspace
from statistics import mean
from math import sqrt, floor, copysign
import orix

from spatialDecompFunctions import householderMatrix, translationMatrix, erase_linearly_dependent_points

x = X[:,0]
y = X[:,1]

k = ConvexHull(X)
k = erase_linearly_dependent_points(k.simplices)

In [ ]:
k.simplices

In [ ]:
from scipy.spatial import ConvexHull
k = ConvexHull(X)
print(k.vertices)

In [ ]:
from numpy import cumsum
nto = np.array([4,4,4,4,4,4])
cs = cumsum(np.hstack([1, 1 + nto]))

In [ ]:
bound_X

In [ ]:
1 + nto

In [ ]:
np.hstack([1, 1 + nto])

In [ ]:
X = np.array([[0, 0],[156.499511761733, 0],
              [156.749510981800, 0.433011351008059], 
              [156.749510981800,146.79084799],
              [17320.249999220066666,146.790847991732],
              [0, 146.357836640724], 
              [0,0]])
a = np.array([1, 6, 11, 16, 21, 26, 31])
X2 = left_hand_assignment(X,a)

In [ ]:
print(np.round(X2,2))

In [ ]:
edge_direction

In [ ]:
X

In [10]:
'''% shift the starting vertex
bX = squeeze(double(axis2quat(zvector,edgeAngle)* ...
  vector3d([0; radius; 1])));
offsetX = bX - boundingX(1:end-1,:);
'''
import orix.quaternion as quat
r= quat.Rotation.from_axes_angles([0,0,1],np.pi/3.)
#print(r.data)
radius = np.mean( sum( (unit_cell**2) ** 0.5 ))

import orix.vector as vect
a = vect.Vector3d([0, radius, 1])
a.rotate(r)
b_X = np.squeeze(r * a)
print(a, b_X)
#np.vstack([r.data, np.array([[0, radius, 1]])])
#b_X = np.squeeze( np.array([r.data, np.array([[0, radius, 1]]).T], dtype=np.float64) )
#offset_X = b_X - bounding_X[1:-2, :]
print(b_X.data)

Vector3d (1,)
[[0.     1.0773 1.    ]] Vector3d (1,)
[[-0.933   0.5387  1.    ]]
[[-0.93300979  0.53867345  1.        ]]


In [ ]:
import numpy as np
from scipy.spatial import ConvexHull

points = X   # 30 random points in 2-D
hull = ConvexHull(points)

#Get centoid
cx = np.mean(hull.points[hull.vertices,0])
cy = np.mean(hull.points[hull.vertices,1])


boundingX = erase_linearly_dependent_points(X)

tmp = boundingX - [cx, cy]
#print(tmp)


b = np.argsort(np.arctan(tmp[:,1]/tmp[:,0]))

from orix.utilities.utilities import sortrows
a = sortrows(boundingX)
a = np.flipud(a)
bounding_X = np.vstack([a[-1,:], a])
print(boundingX)
edge_direction = np.diff(bounding_X)
#print(edge_direction)
#edge_angle = np.arctan2( edge_direction[:, 1], edge_direction[:,0] )
#edge_length = np.sqrt( (edge_direction**2).sum(axis = 1, keepdims=True))

zz = 
print(zz)



In [ ]:
XV = np.hstack([X, np.ones([X.shape[0],1])])

In [ ]:
print(XV)

In [ ]:
r.data